In [1]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [2]:
def load_glove_model(glove_file_path):
    glove_model = KeyedVectors.load_word2vec_format(glove_file_path, binary=False, no_header=True)
    return glove_model

def get_corpus():
    with open("../data/processed/corpus", "r") as f:
        sentences = f.readlines()
    return sentences

In [3]:
sentences = get_corpus()
tokens = [sent.split() for sent in sentences]

In [4]:
base_model = Word2Vec(vector_size=300, min_count = 20, epochs=20)
base_model.build_vocab(tokens)
total_examples = base_model.corpus_count

In [5]:
base_model.wv.most_similar('#person1#', topn=10)

[('will,', 0.19918687641620636),
 ('big,', 0.18685618042945862),
 ('across', 0.1850905865430832),
 ('admires', 0.1844509094953537),
 ('april', 0.17851804196834564),
 ('break.', 0.17759983241558075),
 ('article', 0.17517882585525513),
 ('no', 0.17481420934200287),
 ('fever.', 0.17363373935222626),
 ('(', 0.17225077748298645)]

In [6]:
corpus_path = '../embeds/GloVe/glove.corpus.300d.txt'
corpus_model = load_glove_model(corpus_path)
base_model.build_vocab([list(corpus_model.key_to_index.keys())], update=True)

In [7]:
corpus_model.most_similar('#person1#', topn=10)

[('tells', 0.668127179145813),
 ('asks', 0.6325467228889465),
 ('#person2#', 0.5329562425613403),
 ('wants', 0.5291920304298401),
 ('suggests', 0.5121132135391235),
 ('thinks', 0.5016119480133057),
 ('recommends', 0.49962735176086426),
 ('<sos>', 0.4889325201511383),
 ('advises', 0.4694846272468567),
 ('complains', 0.4649205505847931)]

In [8]:
base_model.train(tokens, total_examples=total_examples, epochs=base_model.epochs)
base_model_wv = base_model.wv


In [9]:
base_model_wv.most_similar('#person1#', topn=10)

[('#person2#', 0.7208516597747803),
 ('sam', 0.6329665184020996),
 ('mike', 0.5963233709335327),
 ('#person3#', 0.5918983221054077),
 ('jeff', 0.5866713523864746),
 ('mary', 0.5842511653900146),
 ('alice', 0.5810478925704956),
 ('jack', 0.5808283090591431),
 ('julie', 0.5800818204879761),
 ('amy', 0.577274739742279)]

In [37]:
base_model_wv.save_word2vec_format('../models/GloVe-Word2Vec/glove.bin')